In [ ]:
import warnings
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
warnings.filterwarnings('ignore')  # Игнорировать все предупреждения (не рекомендуется в продакшн-коде)
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from numpy import mean
from numpy import var
from math import sqrt
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from causalinference import CausalModel
from scipy.stats import chisquare
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.power import tt_ind_solve_power
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from causalinference import CausalModel
import help_tools as ht

### Разбор статей и книг

#### Avito Habr: лайфхаки по улучшению АБ тестов  

https://habr.com/ru/companies/avito/articles/571094/  
https://habr.com/ru/companies/avito/articles/571096/  
https://habr.com/ru/companies/avito/articles/590105/

---
1. Для АА-валидации критериев можно использовать исторические данные с метрикой (например, выручка), поделив  
ее на множество сегментов (например, группы товаров/регион/сайт итд): каждый сегмент = АА тест с реальной метрикой.  
Каждый такой тест можно превратить в AB - симулируя смещение одной из веток. Моделируем аплифт, сохраняем real std.
2. При фильтрации выбросов корректно убрать топ N% наиболее крупных клиентов - трешхолд общий для теста и контроля.  
outlier_threshold = np.quantile(np.concatenate(control_before, test_before), 0.99)  
Так избавляемся от смещения если оценивать его по control/test выборкам.
3. При анализе AB относительные дов. интервалы типа 10+-5% нагляднее чем абсолютные или расчет p_value.  
Аналогично при расчете чувствительности можно вместо MDE использовать оценку ширины дов. интервала.  
Ширина дов. интервала разности средних двух выборок схожих дисперсий half_w = sqrt(2) * MDE.  
4. CUPED/CUPAC: берем пред-тестовый датасет - обучаем на нем модель, предсказываем на тестовых участниках  
метрику -> оцениваем снижение дисперсии (aka дов интервалов). Меняя модели и гиперпараметры - подбираем оптимальный Y_forecast.  
PS. нормируем признаки для удобства - StandardScaler.  
5. Для фильтрации признаков, на которых идет обучение для CUPAC - жадный алгоритм добавления/удаления признаков (например, SequentialFeatureSelector из библиотеки scikit-learn на Python). Более быстрые алгоритмы - на основе  
feature importance (SelectFromModel из библиотеки scikit-learn на Python). Можно также использовать add-del  
функциональность - добавляем/удаляем признаки пока не получим максимального улучшения качества модели 

#### Kohavi book: Trustworthy online controlled experiments

*Randomized controlled experiments are the gold standard for establishing **causality***

<u>When we can use AB testing</u>

- there are units (ex. users) that can be assigment to variants
- there are enough users for testing (>10**3)
- key metric (OEC) could be evaluated and have sensitivity
- changes are easy to make

**Best practise**: using AB testing together with Ajile software development process & Lean
AB testing can be automated and using together with CI/CD processes 
Important target - to make one iteration from delevopment to testing fater
In MSFT only 30% of test’s successful. In well-optimized domains like Google rate ~ 10-20%
Most of your work will be rejected - build your process according this point
`"If you have to kiss a lot of frogs to find a prince, 
find more frogs and kiss them faster and faster"`

h_0: hypothesis that control and experiment values are the same
p_value: the probability of getting this or more extreme observed result if H_0 is True 

<u>Internal validity</u>

There is a trustworthiness of conclusions on current data 
- sample ratio mismatch problem (health checks)
- statistical tests suitability (AA-validation, p_val pick problem, filter outliers)
- residual effect (SRM ⇒ when test was cracked at first and we have affected users)
- SUTVA (units are independent and also control/test are separate from each other)

<u>External validity</u>

Is it possible to apply test results to more general cases.

- is it okey on new regions, platforms etc. (generalisation)
**Solution**:  rerun test in new condition

- long term effects (can we generalize conclusion across time)
   long term ⇒ primacy (old) and novelty (new) effects

**Solution:** check treatment effect with coinfidence interval across the time
Also, is possible to fix cohort of users coming in exp first days and track effect(time)

<u>Segmentation </u>
(additional sources on page 122)
If segm_1 goes up and segm_2 goes up - (total_1_2 can goes any direction)
Because of moving users between segments

If segments area different: s1_control, s2_control, s3_exp, s4_exp - it could 
be Sympson’s paradox (`a/b<A/B and c/d<C/D but (a+c)/(b+d)>(A+C)/(B+D)`)
Example: 44/700 & 46/700 VS 75/1200 & 17/300

Be careful aggregating data collected at different percentages
Both cases is about **non representative** groups in control and experiment.

<u>Metrics taxanomy</u>

**Goal metrics** - metrics for your company mission. Why does your product exist
At first, you can articulate your main goal in words. There are small set of long term metrics.

**Driver metrics** - surrogate, predictive metrics. They have good sensitivity, short term behaviour.
There is not about what is success, but about way (action) to achive this
Example: AARRR, HEART, funnels frameworks

**Guardrail metrics** - internal validity check metrics and system protection metrics.
We don’t want to drop this metrics improving goal and driver ones

<u>Metrics evaluation</u>

Important part of ab-evaluation of metrics - to proove causality between driver metrics and goal metrics (ex. LTV could been evaluated periodically to check if errors stay small)

There are few points how to establish causal links between metrics:

- use surveys, focus groups e.t.c to check whether they all point in same direction
- use observational data (logs e.t.c) to check your hypotheses
- check whether similar validation is done at other companies
- conduct an experiment with primary goal of evaluating metrics
- use historical experiments to check for sensitivity and causal alignment

It is not enough to be agile and to measure
You must make sure your metrics guide you in the right direction

*Literature*:
1) Spitzer, Dean R. 2007. Transforming Performance Measurement: Rethinking the Way We Measure and Drive Organizational Success. AMACOM
2) Parmenter, David. 2015. Key Performance Indicators: Developing, Implementing, and Using Winning KPIs. 3rd edition. John Wiley & Sons, Inc
3) McChesney, Chris, Sean Covey, and Jim Huling. 2012. The 4 Disciplines of Execution: Achieving Your Wildly Important Goals.

<u>Metrics for experimentation</u>

Not all metrics that are used for business reporting purposes are appropriate for experimentation

Some requirements for good experimentation metrics:

- measurable (actually, with numbers eq)
- sensitivity - you can detect meaningful changes during necessary testing period
- attributable (you can assign this metric to testing variants = branches)

<u>Overall evaluation criteria</u>

In AB-tests you need to optimise several metrics simultaneously (ex. monetary and engagement) 
OEC = numerical model of trade-offs between different metrics (ex. FICO-score on risk card)

Assume, you have N metrics: metric_1, … metric_N
metrics → normalisation to {0;1} range → OEC = weighted sum (w1 * metric1 + w2 * metrics2 …)

Next step - minimise the number of metrics, using prioritisation. Good number is ≤ 5
It emphasize you on main metrics and helps with multiple comparison problem.

*Example: Amazon’s optimisation of sending emails to customers*
OEC = sum(Revenue_customer - unsubcribes_count * Lifetime_loss) / N_customers
revenue - profit with email using
unsubcribes_count - count of users who delete subcription because too much emails
lifetime_loss -  LTV of churn users who delete subcription

<u>Complementary and Alternative Techniques to Controlled Experiments</u>

UX interviews, surveys, focus groups, logs analysing - methods to understand customers problems. You can use this researches to generate more ideas for ab testing, create causalities hypothesis about metrics e.t.c

<u>Observational Causal Studies</u>
(more details in Stats/causal inference)

Exp_effect = Outcome_for_treated - Outcome_for_untreated = 
(Outcome_for_treated  - Outcome_for_treated_if_not_treated) + (Outcome_for_treated_if_not_treated - Outcome_for_untreated) = 
Impact_of_treatment_on_treated + Selection_bias

In AB-tests `Selection_bias = 0 → Exp_effect = Impact of treatment`
But sometimes we can’t run ab-tests. We can use Observational Causal Studies

Main questions:

- how to construct control and treatment groups for comparison
- how to model the impact on treatment group

<u>The statistics in Online controlled Experiments</u>

With normal distribution of (M2 - M1) we can use T-test. M - population mean.
T = delta_M / sqrt(var(delta_M)) = delta_M / sqrt(var(M1) + var(M2))
var(M) = var(population) / N

`p_value = P(delta_observed | Ho=True)`
Using Bayes: P(Ho=True | delta_observed) = p_value * P(Ho=True) / P(delta_observed)

Main assumption: T-statistic has normal distribution
M → Normal_dist (CLT)
Minimum number of samples ⇒ depend of **skewness** coef. of sample sitribution
`N_min ~ 355 * s**2; s = (M(Y - M(Y)))**3 / (Var(Y))**(3/2)`

**Power** is the probability of detecting a difference between the variants , that is, rejecting the null, when there really is a difference. `Power = 1 - Type_2_error`

With power=80%, alpha=5% and mean normal distribution, we can calc N samples:
N ~ 16 * var(Y) / mde**2; mde = min_detective(M2 - M1)

<u>Multiple Testing</u>

When you have too much metrics (K metrics = K_per_exp * N_experiments)
chance of random  “success” increased like `alpha_rand = K * alpha`

Bonferroni adjustment: p_val ⇒ p_val / K. That’s too conservative.

Common pipeline:
- first-order metrics (expect to be impacted of experiment)
- second-order metrics (potentially to be impacted - example, cannibalization)
- third-order metrics (unlikely to be impacted)

Use different p_value in groups: p_val_1 = 0.05; p_val_2 = 0.01; p_val_3 = 0.001
**The stronger you believe that Ho is True - lower significance you should use**

<u>Long and short term effects</u>

Why long and short is differ:

- user learned effects (positive or negative)
- network effect or effect of resourses constraints (ex. num of cars, applicants etc)
- delays between experience and target actions
- sesonality, competitive
- retention effects vs clickbaits and so on

Methods to estimate long term effects:

- **cohort analysis** (if cohort is representative to all traffic - it would be good proxy)
You can use additional methods to improve the generalizability, such as a weighting adjustment based on stratification
- **holdback experiments** (10% of users which use only Control version during several months)

### Теория

https://habr.com/ru/users/nnazarov/ - полезный цикл статей

---
Как описывали в Causal Inference, при рандомизированном разбиении выборки ничего не влияет на T.  
В результате ATE = ATT; Влияние конфаундеров исчезает.  
При этом остаются ряд других проблем связанных с оценкой эффекта через статистические тесты:  

1. Проблема точности оценки ATE и связанная с ней длительность теста. См подробнее раздел по MDE 
2. Проблема применимости стат-теста к конкретной метрике (см подробнее раздел про Валидацию, сбалансированность)
3. Проблемы множественных сравнений и работы с Ratio-метриками (линеаризация)

#### Variance reduction. CUPAC, CUPED, Stratification

https://medium.com/glovo-engineering/variance-reduction-in-experiments-using-covariate-adjustment-techniques-717b1e450185  
https://github.com/Glovo/covariate-adjustment-blogpost/tree/main  
https://habr.com/ru/companies/X5Tech/articles/780270/

---
Из раздела "Линейная регрессия" (Causal inference) - в системе Y ~ T + X учет ковариат X, которые НЕ влияют на T  
позволяет снижать дисперсию метрики Y -> увеличивать точность оценки ATE (по сути коэф. перед T = treatment).   
Но это можно рассмотреть и так: Y - X = Y_adj ~ T; Y_adj = скорректированная по дисперсии метрика к Y.  
Основные условия:
1. avg(Y) = avg(Y_adj)
2. std(Y_adj) < std(Y)
3. Y ~ Y_adj (сонаправленность метрик)
  
Пусть у нас есть ковариата X, для которой cov(X, Y) > 0 при этом cov(X, T) = 0  
Тогда согласно п.3: Y_predict = Yp = a0 + tetta * X , по методу наименьших квадратов tetta = cov(X,Yp) / var(X)  
Рассмотрим Y_adj = Y - Yp + avg(Yp):
видно что avg(Y_adj) = avg(Y)  
т.к X объясняет часть вариации Y -> std(Y_adj) < std(Y)  
Y_adj ~ Y по методу построения Yp.  (К слову, рандомная X не подходит, так как будет плохо предсказывать Y в регрессии)
  
С точки зрения длительности AB: N ~ var(X)/mde^2(X) -> уменьшая дисперсию метрики, можно пропорционально снижать N.
  
<u>CUPED = Controlled-experiment Using Pre-Experiment Data</u>  
Если в качестве X использовать Y_prev = значения метрики на пред-экспериментальном периоде (до влияния T), то  
Y_adj = Y_cuped = Y - Yp + avg(Yp) = Y - tetta * (Y_prev - avg(Y_prev))  
Пре-экспериментальный период никак не может повлиять на T -> условие выполняется.  
Если брать не очень большой период (например, последние 2 недели) -> достигается адекватная cov(Y, Y_prev).  
  
<u>Мультиковариатная регрессия</u>  
CUPED - частный случай, когда мы используем Y_prev как ковариату-предиктор для Y в уравнении Y ~ T + Y_prev  
Если мы знаем другие предикторы, уточняющие поведение Y, которые НЕ влияют на T, то Y ~ T + X1 + X2 + ...  
Учет таких предикторов повышает точность оценки ATE (см ниже пример в коде)
  
<u>CUPAC = Control Using Predictors as Covariates</u>  
CUPAC - расширение над уравнением регрессии с использованием ML, когда для снижения вариации целевой метрики Y  
используются модели вроде случ. лесов итд.  
Идея: имеем ковариаты X1, X2, ... Y_prev -> обучаем модель ML(X1, X2 .., Y_prev) -> Y_forecast.  
Используем Y_forecast как ключевую ковариату в уравнении Y ~ T + Y_forecast.  
PS. CUPED - частный случай CUPAC с одной ковариатой Y_prev   
PSS. Мультирегрессия - частный случай CUPAC где ML-модель - линейная регрессия.  
<u>Во всех</u> случаях Y_adj = Y - tetta * (Y_forecast - avg(Y_forecast))  
std(Y_adj) < std(Y); avg(Y_adj) = avg(Y)  
    
<u>Подбор ковариат и пред-тестового периода</u>      
Главное условие: ковариаты Y_prev, X1, X2, ... - никак не влияют на распределение T и обратно: cov(Y_forecast, T) = 0  
Поэтому их берут на пред-тестовом периоде, чтобы Y_forecast = ML.fit(X1, ...) точно не зависело от последующего T.  
   
При выборе пред-тестового периода можно использовать знания о сезонности (искать аналогичный сезон в прошлом).  
Размер пред-тестового периода может варьироваться (от exp_duration и больше)
Подбор пред-тестового периода можно совершать, каждый раз оценивая насколько Y_adj(Y_forecast) снижает дисперсию относительно Y. 
  
CUPAC/CUPED также можно использовать для бинарных метрик (вроде конверсии) - он будет улучшать чувствительность оценок.  

In [ ]:
# генерация данных
n = 5000
data = pd.DataFrame({
    'T': np.random.binomial(1, 0.5, size=n),
    'Y': np.random.normal(size=n),
    'Y_prev': np.random.normal(size=n),
    'Cov2': np.random.normal(size=n),
})
effect = 2

# пре-тестовый период (нет зависимости T так как нет еще теста)
data_prev = data[:2500] 
# data_prev['Y'] += 3 * data_prev['Y_prev'] + 4 * data_prev['Cov2']
data_prev['Y'] += 3 * data_prev['Y_prev'] + 4 * data_prev['Cov2']**2


# тестовый период
data_now = data[2500:]
# data_now['Y'] += effect * data_now['T'] + 3 * data_now['Y_prev'] + 4 * data_now['Cov2'] 
data_now['Y'] += effect * data_now['T'] + 3 * data_now['Y_prev'] + 4 * data_now['Cov2']**2 


# реализация CUPED
# в AB тесте оценку tetta можно проводить как по now, так и по prev данным.
# если размеры AB групп достаточно велики, лучше оценивать tetta на объединённых данных контрольной и экспериментальной групп. 
# Если размеры групп малы, то оценка tetta на исторических данных может дать лучшее качество.
def get_cuped_adj(Y, Y_prev):
    # Y = целевая метрика; Y_prev эта же метрика на клиентах на пред-тестовом периоде
    tetta = np.cov(Y, Y_prev)[0][1] / np.var(Y_prev)
    return Y - (Y_prev - np.mean(Y_prev)) * tetta

# оценка ATE через регрессию
def get_est(formula, df, text):
    model = smf.ols(formula, data=df).fit()
    conf = np.round(model.conf_int().loc['T'], 3); std = round(model.bse.loc['T'], 3)
    print(f'{text}: ate = {conf[0]} - {conf[1]}; std_ate = {std}')
    
# Пример использования CUPAC: получаем y_forecast = ML(X1, X2 ...) через древесный регрессом; корректируем Y
def get_cupac_adj(X_past, Y_past, X, Y):
    # X_past, Y_past - фичи и целевая метрика на предэкспериментальном периоде
    # X, Y - фичи и метрика в эксперименте - для предикта Y_forecast и поправки
    gbm = HistGradientBoostingRegressor().fit(X_past, Y_past)
    Y_forecast = gbm.predict(X)
    return get_cuped_adj(Y, Y_forecast)


Y_cuped = get_cuped_adj(data_now.Y, data_now.Y_prev)
Y_cupac = get_cupac_adj(
                      data_prev[['Y_prev', 'Cov2']].values, # целевая метрика пред-тестового периода
                      data_prev.Y.values, # целевая метрика пред-тестового периода
                      data_now[['Y_prev', 'Cov2']].values,
                      data_now.Y.values
                     )

data_now['Y_cuped'] = Y_cuped
data_now['Y_cupac'] = Y_cupac

print(f'реальный ATE = {effect}')
get_est('Y ~ T', data_now, 'без редукции VAR')
get_est('Y_cuped ~ T', data_now, 'CUPED - формула')
get_est('Y ~ T + Y_prev', data_now, 'CUPED, как частный случ. мультиковар. регрессии')
get_est('Y ~ T + Y_prev + Cov2', data_now, 'Мультиковар. регрессия')
get_est('Y_cupac ~ T', data_now, 'CUPAC обученный на prev-периоде')

<u>STRATIFICATION</u>  
https://habr.com/ru/companies/yandex/articles/497804/  
https://habr.com/ru/companies/X5Tech/articles/596279/  
https://habr.com/ru/companies/X5Tech/articles/826488/  

---
<u>Ключевая идея</u>    
заменяем среднее для метрики X на средневзвешенное по стратам.
Страта - группа с одинаковым значением S = комбинированной ковариаты страты.  
  
Например, Y - сумма покупок компаний; X - сегмент бизнеса, возраст компании; S = {SMB_new, SMB_old, Large_new, ....}.  
При этом SMB_new, SMB_old ... это конкретные страты, по которым будем взвешивать среднее.  
Для составления вектора S можно использовать как простую конкантенацию так и kNN методы.  
  
Веса каждом страты W_s = len(X in S) / len(X) = вероятности, оцениваются на пред-экспериментальном периоде.  
Среднее тогда avg_strat(X) = sum(W_s * avg(X in S)); std_strat_X < std_X.  
Причина снижения дисперсии - фильтрация меж-стратовой дисперсии (см "Статистика") - остается только внутри-стратовая.  
Снижение тем выше - <u>чем сильнее разница средних между стратами</u> (то есть выше межгрупповая дисперсия)
  
Важное условие: cov(S, T) = 0, метрика S не влияет на экспериментальную разбивку, существует до и после начала теста.  
Сам тест также не влияет на метрику S.
  
Стратификация классическая и пост-стратификация:
1. Классическая - когда мы семплируем A/B группы следя за равномерным наполнением страт и затем применяем avg_strat_ttest  
2. Пост - когда мы семплируем без контроля равномерности страт (как обычно), но далее также применяем avg_strat_ttest  
Различия в оценках std методами 1 и 2 стремятся к нулю при большом кол-ве данных (>1e3) -> обычно можно идти по пути 2.
  
Выбор признаков для S можно осуществлять на исторических данных исходя из условия:  
максимальная меж-групповая дисперсия среднего целевой метрики посчитанной по разным стратам.  

---
СUPAC vs Stratification.  
В большинстве случаев, если использовать S в CUPAC для предикта Y_forecast, то снижение  
дисперсии тотал метрики за счет обнуления межгрупповой дисперсии уже будет учтено в алгоритме.  
Поэтому использование стратификации поверх CUPAC не даст значительного улучшения.  
  
Исключение - если после старта эксперимента проихошло <u>не зависящее от эксперимента</u>  влияние на метрику Y  
конкретно в каких-то стратах (например, сезонный рост покупок в сегменте LARGE которого не было на пред-периоде).  
В этом случае, Y_forecast не учтет выросшую в тесте межгрупповую дисперсию, а методы стратификации поверх CUPAC  
смогут ее контролировать. Поэтому в общем случае полезно использовать оба метода.  

In [ ]:
# сгенерируем тестовые данные; Y будет зависеть от значения страт
# чем больше разброс id страт - тем больше разброс Y_strat - лучше работает стратификация
size = 5000
effect = 0.01
std = 3
strat_seq = [1, 2, 5]

# распределение страт на пред-экспериментальном периоде
df_prev = pd.DataFrame({'S' : np.random.choice(strat_seq, size=size)})
weights = df_prev.S.value_counts(normalize=True) # series

df_con = pd.DataFrame({'S' : np.random.choice(strat_seq, size=size)})
df_con['Y'] = df_con['S'].apply(lambda x: np.random.normal(x**2, std))

df_test = pd.DataFrame({'S' : np.random.choice(strat_seq, size=size)})
df_test['Y'] = df_test['S'].apply(lambda x: np.random.normal(x**2 + effect, std))

conf1 = ht.ttest_stratification_calc(df_con, df_test, weights)[0] # пост стратификация
conf2 = ht.ttest_calc(df_con.Y, df_test.Y)[0] # классический тест
print(f"""conf_width: classic test = {round(conf2[1] - conf2[0], 4)}; stratification = {round(conf1[1] - conf1[0], 4)}""")

#### Длительность теста и MDE 

MDE = Minimum Detectable Effect = минимальное изменение которое будет детектируемо через стат-тест с  
выбранными уровнями ошибки первого и второго рода.  
  
В случае с T-тестом (см страничку "Статистика") оцениваем delta = avg(control) - avg(experiment).  
Дисперсия средневыборочного std_X_mean = std(X)/sqrt(N), N - длина выборки.  
Поэтому мин. детектируемая delta средних: mde = T(alpha, power) * std(X) / sqrt(N).  
  
Отсюда следует, что N (кол-во клиентов aka длительность теста) ~ var(X) / mde^2.  
Чем меньше var(X) = шум метрики - тем меньше данных надо для фиксированной точности.  
Кроме того с ростом ожидаемого mde квадратично падает N
  
1) Учитывая в дизайне АБ теста рост mde - можно квадратично ускорять его.  
Пример: замеряем как изменится рост выручки X от роста цены на 10%.  
Если запустить тест в формате (уменьшили цену на 10%) VS (увеличили цену на 10%), то  
ожидаемая дельта эффекта удвоится -> данных надо будет гораздо меньше.  
  
2) Другой вариант - сокращать var(X) - см подробнее в разделе "Variance reduction"  

Приложение:  
При оценке mde эксперимента на контрольной выборке, полагаем что std_experiment = std_control.  
В будущем, на накопленных данных control/experiment можно посчитать mde_real(control, experiment).  
Тогда в формуле std будет учитываться как sqrt(std_control^2 + std_experiment^2)

In [ ]:
control, test = np.random.normal(1, 1, 1000), np.random.normal(1.2, 1, 1000)
ht.ttest_calc(control, test) # confint; pvalue

In [ ]:
# пример оценки MDE
control = np.random.normal(1, 1, 1000)
ht.get_mde_detail(control)

#### Сбалансированность и репрезентативность

1. Репрезентативность - обобщаемость используемой выборки на всю генеральную совокупность.   
При нестационарном поведении генеральной совокупности во времени необходимо случайно  
семплировать выборку для теста также по времени.   
2. Сбалансированность - условия по возможности попадания семплов в выборку в тесте/контроле должны быть одинаковые  
Перекос в размере выборки относительно ожидаемой показывает на ошибкув рандомизации.  
  
---
Тест на сбалансированность через хи-квадрат:  
  
Критерий соответствия эмпирических частот ожидаемым.  
При сравнении номинативных признаков образуется таблица сопряженности с совместными частотами.  
Она же образуется, если сравниваем теоретическое и реальное распределения вероятностей.  
Частный случай таблицы - вектор эмпирических частот признака (например частоты орла/решки).  

H0-гипотеза: наблюдаемые частоты совпадают с ожидаемыми (теоретическими).  
В этом случае chi^2 = sum((Obs - Exp)^2 / Exp) -- расстояние Пирсона.  
Для таблицы размерностью ij критерий будет удовлетворять df = (i-1)(j-1) степень свободы.  
Так как мы знаем тотал значения таблицы и значит одна степень свободы всегда фиксирована.  

PS. Величина Obs распределена биномиально (вероятность попасть в ячейку j Obs раз).  
При малых вероятностях получаем Пуассона, где std^2=mean=Exp при нулевой гипотезе. Тогда t-score  
(Obs - Mean) / Std ~ (Obs - Exp)/sqrt(Exp) ~ N(0, 1).

In [ ]:
# Оценка сбалансированности выборок в АБ тесте
f_real = [100, 110, 105]
print('Три ветки, ожидаемое распределение равномерно: ', ht.check_branch_balance(f_real))
f_real = [85, 20]; f_exp = [80, 20]      
print('Две ветки, катим тест 80/20: ', ht.check_branch_balance(f_real, f_exp))
f_real = [100, 20]; f_exp = [80, 20]      
print('Две ветки, катим тест 80/20: ', ht.check_branch_balance(f_real, f_exp))

#### Множественные сравнения

https://habr.com/ru/articles/772940/
  
При проведении A/B/C... = N тестов могут возникнуть такие задачи:
1. Определить ветку-победителя. Потребуется сделать K = N * (N-1) / 2 стат-тестов, проверяя каждую пару веток.
2. Найти хотя бы одну ветку, лучшую чем контроль. Здесь требуется совершить N сравнений с контролем.  
3. Остановить тест если хотя бы одна из K значимых метрик прокрасится отрицательно (даже в простом A/B тесте)

Проблема в контроле ошибки первого рода: если не использовать поправки на множественное сравнение,  
то например вероятность найти хотя бы один прокрас FWER = 1 - (1 - alpha)^N ~ alpha * N >> alpha 

В зависимости от консервативности контроля выделяют два типа множественных корректировок:

1. Контроль FWER = family-wise error rate = вероятность совершить хотя бы одну ошибку при проверке X тестов.  
Хотя бы одну ошибку сделали = сделали неправильное решение на базе группы.  
Считаем любую ошибку критичной и ожидаем ее появление в небольшое кол-ве случаев.  
Используют поправку Бонферрони (alpha_adj = alpha / N); поправку Холма (более мощная), Тьюкки итд.  
По мощности оптимально использовать метод Холма

2. Контроль FDR = false discovery rate = доля всех ложных прокрасов среди тестов, которые <u>прокрасились</u>  
Здесь допускаем чтобы из K прокрашенных тестов у нас было alpha% прокрашенных по ошибке.  
Подходит, когда фильтруем интересные инсайты и допускаем наличие среди них доли ложно-прокрашенных.  
Мощность в среднем, очевидно, выше чем при контроле FWER
Здесь по мощности оптимально использовать метод Бенджамини-Хохберга  

Чем эффективнее идет контроль FWER (шанс хотя бы одного ложного прокраса), тем хуже чувствительность, например  
метрика FWER-II (шанс хотя бы одного случая, когда не заметили реальный прокрас).  

In [ ]:
fwer_list, tn_list = [], []
fwer_list2, tn_list2 = [], []
fwer_list3, tn_list3 = [], []
fwer_list4, tn_list4 = [], []

def get_effect(real_effect, p_val_list):
    fp = [] # ошибка первого рода
    tn = [] # ошибка второго рода
    for idx, p_val in enumerate(p_val_list):
        if real_effect[idx] == 0 and p_val < 0.05: # FP
            fp.append(1); tn.append(0)
        elif real_effect[idx] == 0 and p_val >= 0.05: # FN
            fp.append(0); tn.append(0)
        elif real_effect[idx] == 1 and p_val < 0.05: # TP
            fp.append(0); tn.append(0)
        elif real_effect[idx] == 1 and p_val >= 0.05: # TN
            fp.append(0); tn.append(1)
            
    fwer = max(fp) # хотя бы один ложный прокрас в группе
    fwer_II = max(tn) # не нашли хотя бы один реальный прокрас в группе
    return fwer, fwer_II

for _ in range(1000):
    
    # генерируем одинаковые ветки
    test_list = []
    for _ in range(3): # кол-во веток
        test_list.append(np.random.normal(1, 1, 200)) # mu, std, size
        
    # добавляем ветку с реальным эффектом
    eff_delta = 0.3
    test_list.append(np.random.normal(1 + eff_delta, 1, 200))
    idx_last = len(test_list) - 1
    

    # попарное вычисление тестов
    p_val_list = []
    real_effect = []
    for idx, test in enumerate(test_list):
        for idx2, test2 in enumerate(test_list):
            if idx > idx2:
                p_val_list.append(stats.ttest_ind(test, test2)[1])
                
                # помечаем если имеется реальное различие в тестах
                if idx == idx_last or idx2 == idx_last:
                    real_effect.append(1)
                else:
                    real_effect.append(0)
                
    # корректировки p_val для множественного сравнения
    p_val_bonf = multipletests(p_val_list, alpha = 0.05, method='bonferroni')[1] # holm
    p_val_holm = multipletests(p_val_list, alpha = 0.05, method='holm')[1] # holm
    p_val_bh = multipletests(p_val_list, alpha = 0.05, method='fdr_bh')[1] # Benjamini/Hochberg 

    # принятие решения по группе - расчет коллективного ложного прокраса и ошибки 2-го рода
    fwer, fwer_II = get_effect(real_effect, p_val_list)
    fwer_list.append(fwer); tn_list.append(fwer_II)
    
    fwer, fwer_II = get_effect(real_effect, p_val_bonf)
    fwer_list2.append(fwer); tn_list2.append(fwer_II)
    
    fwer, fwer_II = get_effect(real_effect, p_val_holm)
    fwer_list3.append(fwer); tn_list3.append(fwer_II)
    
    fwer, fwer_II = get_effect(real_effect, p_val_bh)
    fwer_list4.append(fwer); tn_list4.append(fwer_II)

# статистики для разных подходов
print(f'No correction. FWER = {np.mean(fwer_list)}; FWER-II = {round(np.mean(tn_list), 5)}')
print(f'Bonferroni-FWER correction. FWER = {np.mean(fwer_list2)}; FWER-II = {round(np.mean(tn_list2), 5)}')
print(f'Holm-FWER correction. FWER = {np.mean(fwer_list3)}; FWER-II = {round(np.mean(tn_list3), 5)}')
print(f'BH-FDR correction. FWER = {np.mean(fwer_list4)}; FWER-II = {round(np.mean(tn_list4), 5)}')

#### Валидация

https://habr.com/ru/companies/hh/articles/321386/  
https://habr.com/ru/companies/X5Tech/articles/706388/

---
Когда мы работаем с метрикой X и планируем оценивать ее изменение в AB тесте, используя статистический тест,  
нужно перед этим убедиться, что пройдена валидация метрики для этого теста.  
  
Валидация:
убеждаемся, что при выбранной для теста ошибке первого рода - на группе АА-тестов доля ложных  
прокрасов (FPR) соответствует этой ошибке.  
PS. В общем случае через АА тест можно проверить любой тип теста (Манна-Уитни, bootstrap итд)    
Подробнее рассмотрим T-test на средних, как наиболее часто используемый.  
  
Ситуация, когда метрика не проходит валидацию:
1. Выбросы в метрике -> не выполняется ЦПТ -> средневыборочное не распределено нормально (см "Статистика")
2. Семплы зависимы -> не выполняются классические операции над вероятностями -> нарушение нормальности
3. Слишком мало данных
  
При валидации T-теста, мы разбиваем выборку на случайные пары АА-фрагментов, вычисляем p_val и оцениваем  
долю ложных прокрасов с доверительным интервалом.  
Дополнительно можно посмотреть на нормальность средневыборочного (необходимое условие валидации, но не достаточное)

In [ ]:
# пример адекватной для T-теста метрики
X = np.random.normal(1, 1, 1000) 
ans, p_val_list, avg_list = ht.validation_ttest(X)
print(ans)

In [ ]:
# пример метрики с дополнительными "выбросами"
X = np.append(np.random.normal(1, 1, 1000), np.random.normal(1000, 1, 9)) 
ans, p_val_list, avg_list = ht.validation_ttest(X)
print(ans)

#### Последовательный анализ, бакетный и децильный анализы TODO

In [ ]:
# todo
# https://www.youtube.com/watch?v=p_5YzShN4sg
# децильное распределение val_list1, val_list2 на основании бутстрапа
# N-й дециль -> доверительный интервал , контрольное значение

#### Ratio-метрики. Дельта метод

https://habr.com/ru/companies/X5Tech/articles/740476/
https://habr.com/ru/company/avito/blog/454164/

---

Метрика типа R = X / Y (например средняя длительность сессии на сайте по всем клиентам).  
R = (x1 + x2 + ...) / (y1 + y2 + ...)  
В общем случае x1, x2, ... зависимые, так как один клиент может создавать несколько сессий и пр.  
Поэтому, нельзя работать с параметрическими тестами вроде t-test.  
(зависимость -> не работают теоремы сложения/умножения вероятностей -> не работают все распределения).  
  
1. Чтобы начали работать классические тесты - необходимо превратить Ratio-метрику в поюзерную  
R = X / Y = alpha * X + betta * Y + gamma  
Поюзерная метрика -> пренебрегая зависимостью между юзерами -> подходит под i.i.e
Один из вариантов такого разложения - ряд Тейлора в точке avg(X)/avg(Y). 
Можно разложить до первого члена (линеаризация) или взять больше точности и след члены ряда.
  
2. Можно использовать bootstrap для семплирования Ratio  
Для него нет проблем с тем что распределение не нормальное, но это вычислительно дорого  
  
3. Есть дельта-метод оценки дисперсии метрики X/Y (см статью)  
Он считается быстро и удобен в оценке критериев, но не дает поюзерную метрику с которой потом удобно работать,  
например увеличивая чувствительность и пр.

---
<u>Дельта-метод</u>  
https://www.linkedin.com/pulse/дельта-метод-от-а-до-я-bogdan-pilyavets/?originalSubdomain=ru  
Ключевая идея: если функция g(X) асимптотически нормально распределена по X, то  
ее можно представить в виде g -> (n inf) -> g(X0) + (X - X0) * dg/dx ~ N(0, std)  
Аналогичное разложение работает в многомерном сценарии g(X,Y,Z...) в ряд Тейлора.  
std зависит от производных функции g в опорной точке 0.
Доказательство = разложение в ряд Тейлора + применение ЦПТ.  
  
Частный случай: g = g(X,Y) = X/Y -> avg(X/Y) + dg/dx * (X-avg_X) + dg/dy * (Y-avg_Y).  
Отсюда линеарзиация: разложение ratio-метрики в поюзерную, std(X/Y) ~ std(dg/dx * ...) - считается.    
P.S для метрики X/Y также критично avg_Y > 0; std(X), std(Y) конечны (эффект выбросов не значителен).

In [ ]:
# семплируем отдельных "юзеров" по индексам
def ratio_bootstrap_calc(num_list1, denum_list1, num_list2, denum_list2, iter_=10**4, alpha=0.05):
    boot_list = []
    for _ in range(iter_):   
        idx = np.random.choice(np.arange(len(num_list1)), len(num_list1), replace=True)
        idx2 = np.random.choice(np.arange(len(num_list2)), len(num_list2), replace=True)
        num_list1_, denum_list1_ = num_list1[idx], denum_list1[idx]
        num_list2_, denum_list2_ = num_list2[idx2], denum_list2[idx2]
        delta = sum(num_list2_) / sum(denum_list2_) - sum(num_list1_) / sum(denum_list1_)
        boot_list.append(delta)
    # confint 
    return np.quantile(boot_list, [alpha/2, 1 - alpha/2])

# превращаем метрику в поюзерную раскладывая в ряд тейлора до первого члена (дельта метод)
def ratio_taylor(metric_num, metric_denum):
    """
    X/Y ~ mx/my + 1/my * (X - mx) - mx/my**2 * (Y - my) = mx/my + 1/my(X - Y*mx/my)
    https://habr.com/ru/company/avito/blog/454164/
    """
    mx, my = np.mean(metric_num), np.mean(metric_denum)
    return mx / my + (1 / my) * (metric_num - metric_denum * (mx / my))

In [ ]:
num_list1 = np.random.normal(1, 1, 1000)
denum_list1 = np.random.normal(2, 1, 1000)
num_list2 = np.random.normal(1, 1, 1000)
denum_list2 = np.random.normal(3, 1, 1000)
ratio_bootstrap_calc(num_list1, denum_list1, num_list2, denum_list2)

In [ ]:
linear1 = ratio_taylor(num_list1, denum_list1)
linear2 = ratio_taylor(num_list2, denum_list2)
print('линеаризация + поюзерный ttest дают корректный результат как bootstrap')
ht.ttest_calc(linear1, linear2)[0]